# CONTEXTO
De acordo com uma pesquisa recente da Lending Tree, o roubo em caixas de
autoatendimento é um problema real, e os indivíduos que se envolvem nisso
provavelmente repetirão o comportamento. O Walmart enfrentou perdas com
roubos no varejo, com estimativas de 3 bilhões em 2021, 6,1 bilhões em 2022 e
US$ 6,5 bilhões em 2023, o que mostra um crescimento de $400 milhões de
dólares em perdas por furto no último ano.

Você é o cientista de dados do Walmart trabalhando exclusivamente para a
área de e-commerce. Foi identificado que o maior aumento proporcional em
furtos aconteceu nas compras efetuadas via e-commerce em que usuários
relatam não receberem todos os pedidos feitos nas suas compras. Do
crescimento de roubos de 2022 para 2023, 53% do aumento veio das compras
online.

# OBJETIVO DO PROBLEMA
Você foi designado a cuidar deste projeto para diminuir o número de fraudes e
roubos através dos sites de e-commerce do Walmart. Foi identificado que
estes roubos acontecem na entrega.

Neste projeto, o objetivo principal é identificar possíveis fraudes em entregas
realizadas pelo Walmart na região Central da Flórida para servir de modelo
(caso o resultado do projeto seja positivo) para as demais regiões dos EUA.

# DESCRIÇÃO DO PROBLEMA
Nos EUA o Walmart possui um sistema similar ao Uber em que entregadores se
cadastram para entregar pedidos feitos através do site do Walmart. Estes
entregadores (motoristas) não são funcionários do Walmart, mas trabalham
independentemente aceitando pedidos de entrega e fazendo o recebimento
destes pedidos pela equipe do Walmart e entrega destes pedidos ao endereço
do consumidor. Muitos consumidores relataram que certos itens de seus
pedidos não foram entregues, apesar de o sistema marcar a entrega como
concluída. Isso levanta algumas questões críticas:

1. Fraude do Entregador (Motorista): Há evidências de que motoristas
possam estar reportando a entrega de itens que, na realidade, não
chegaram até o cliente. Eles podem estar omitindo ou desviando itens do
pedido, registrando, no entanto, a entrega total.
2. Erro do Sistema ou Processo: Pode ser que o problema esteja em falhas no
sistema de registro ou no processo de entrega, não se limitando a fraudes
intencionais.
3. Fraude do Consumidor: Em alguns casos, o consumidor pode estar
declarando como não ter recebido um produto que foi entregue para assim
pedir o reembolso do produto.

#### Imports

In [46]:
import pandas as pd

#### Lendo arquivos

In [47]:
df_products = pd.read_csv('Products_Data.csv')
df_customers = pd.read_csv('Customers_Data.csv')
df_drivers = pd.read_csv('Drivers_Data.csv')
df_Missing_Items = pd.read_csv('Missing_Items_Data.csv')
df_Orders = pd.read_csv('Orders.csv')


#### Tratando arquivo PRODUCTS_DATA

In [ ]:
# Função para corrigir a categoria com base no nome do produto
def ajustar_categoria(produto):
    produto = produto.lower()
    
    if any(palavra in produto for palavra in ['milk', 'cheese', 'butter', 'yogurt', 'cream']):
        return 'Dairy'
    elif any(palavra in produto for palavra in ['bread', 'tortilla', 'bun', 'pita', 'crouton']):
        return 'Bakery'
    elif any(palavra in produto for palavra in ['cereal', 'oat', 'granola', 'bar', 'pop-tart']):
        return 'Pantry'
    elif any(palavra in produto for palavra in ['chocolate', 'cookie', 'candy', 'snack', 'cracker', 'pretzel']):
        return 'Snacks'
    elif any(palavra in produto for palavra in ['juice', 'soda', 'coke', 'pepsi', 'gatorade', 'red bull', 'fanta', 'sprite']):
        return 'Beverages'
    elif any(palavra in produto for palavra in ['chicken', 'beef', 'sausage', 'turkey', 'bacon', 'pork']):
        return 'Meat'
    elif any(palavra in produto for palavra in ['laptop', 'phone', 'tv', 'printer', 'camera', 'headphone', 'watch', 'console', 'gaming', 'vacuum', 'drone', 'mouse', 'dell', 'jbl', 'roku', 'nest thermostat', 'fitbit charge 5', 'bose quietcomfort earbuds', 'nvidia geforce rtx 3060', 'sony playstation 5', 'garmin forerunner', 'microsoft surface pro', 'nintendo', 'gopro', 'galaxy', 'google chromecast', 'amazon echo dot', 'apple ipad','apple macbook','apple watch','apple iphone']):
        return 'Electronics'
    elif any(palavra in produto for palavra in ['apple', 'banana', 'orange', 'grape', 'lettuce', 'broccoli', 'spinach', 'vegetable', 'carrot', 'onion', 'garlic', 'pepper', 'potato']):
        return 'Produce'
    elif any(palavra in produto for palavra in ['frozen', 'ice cream', 'waffle', 'pizza', 'shrimp']):
        return 'Frozen'
    elif any(palavra in produto for palavra in ['ketchup', 'mustard', 'sauce', 'pesto', 'hummus']):
        return 'Condiments'
    elif any(palavra in produto for palavra in ['oil', 'flour', 'sugar', 'salt', 'spice', 'baking']):
        return 'Pantry'
    else:
        return 'Supermarket'

# Aplicar a função
df_products['category'] = df_products['product_name'].apply(ajustar_categoria)

# Visualizar o resultado
df_products.head(10)


,produc_id,product_name,category,price
0,PWPX0982761090982,Kellogg's Frosties,Supermarket,$12.53
1,PWPX0982761090983,Uncured Bacon,Meat,$4.67
2,PWPX0982761090984,Whole Milk,Dairy,$9.95
3,PWPX0982761090985,Organic Bananas,Produce,$3.94
4,PWPX0982761090986,Sourdough Bread,Bakery,$19.77
5,PWPX0982761090987,Cheddar Cheese,Dairy,$12.47
6,PWPX0982761090988,Greek Yogurt,Dairy,$18.01
7,PWPX0982761090989,Tomato Soup,Supermarket,$7.82
8,PWPX0982761090990,Frozen Peas,Frozen,$10.10
9,PWPX0982761090991,Honey Nut Cheerios,Supermarket,$17.54


In [49]:
silver1 = pd.merge(df_Orders, df_Missing_Items, on='order_id', how='left')
silver2 = pd.merge(silver1, df_drivers, on='driver_id', how='left')
gold = pd.merge(silver2, df_customers, on='customer_id', how='left')



#### Tratando o arquivo Missing_items

In [50]:
# Empilhando todas as colunas de produtos para contar quantas vezes cada id de produto aparece
all_products = pd.concat([
    df_Missing_Items['product_id_1'],
    df_Missing_Items['product_id_2'],
    df_Missing_Items['product_id_3']
])

# Contando a frequência de cada product_id
product_counts_missing = all_products.value_counts().reset_index()
# fazendo menos 1, pq veio 1 a mais
product_counts_missing['n_missing_products'] = product_counts_missing['count'] - 1
# removendo a coluna count e renomeado a coluna index
product_counts_missing.drop(columns=['count'], inplace=True)
product_counts_missing.rename(columns={'index': 'produc_id'},inplace=True)
product_counts_missing


,produc_id,n_missing_products
0,PWPX0982761090997,18
1,PWPX0982761090993,18
2,PWPX0982761090987,17
3,PWPX0982761091078,16
4,PWPX0982761090983,16
...,...,...
140,PWPX0982761091096,6
141,PWPX0982761091110,6
142,PWPX0982761091115,5
143,PWPX0982761091117,5


In [51]:
gold_products = pd.merge(df_products, product_counts_missing, on='produc_id', how='left')

In [52]:
gold.to_csv('Gold_geral.csv', index=False)
gold_products.to_csv('Gold_products.csv', index=False)